In [1]:
from data.graph_loader import GraphLoader
from data.gtfs_loader import GTFSLoader
from core.routing import RouteService
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx

def openMap(m):
    html = "map.html"
    m.save(html)

    import webbrowser
    webbrowser.open(html)

In [2]:
gtfs_loader = GTFSLoader()
transit_df = gtfs_loader.load_transit_dataframe("data/gtfs")
stops_df = gtfs_loader.load_stops_dataframe("data/gtfs", transit_df)



Loading transit DataFrame from data/gtfs\transit_df.pkl
Loading stops DataFrame from data/gtfs\stops_df.pkl


In [3]:
graph_loader = GraphLoader()
graph_walk = graph_loader.create_graph_walk("data/graphs/ZMG_walk.pkl", "data/osm/ZMG_enclosure_2km.geojson")
graph_transit = graph_loader.create_graph_transit("data/graphs/ZMG_transit.pkl", stops_df)

Loading walking graph from data/graphs/ZMG_walk.pkl
Loading transit graph from data/graphs/ZMG_transit_300.pkl
Loading transit graph from data/graphs/ZMG_transit_300.pkl


In [4]:
route_service = RouteService(graph_walk, graph_transit, stops_df, transit_df)

In [8]:
import random as rd
from shapely.geometry import Point

#choose randomly a node from the largest connected component
def random_conected_walking_nodes():

    connected = False
    attempts = 0

    while not connected:
        # Pick from largest connected component
        start = rd.choice(list(graph_walk.nodes))
        destination = rd.choice(list(graph_walk.nodes))
        
        # Since they're both in the same connected component, they should be connected
        connected = nx.has_path(graph_walk, source=start, target=destination)
        attempts += 1
        print(f"Attempt {attempts}")
        if attempts > 100:  # Much fewer attempts needed now
            raise Exception("Failed to find connected nodes after 100 attempts")
            
    return start, destination

walk_node_start, walk_node_destination = random_conected_walking_nodes()

Attempt 1


In [9]:
import folium

start = Point(graph_walk.nodes[walk_node_start]['x'], graph_walk.nodes[walk_node_start]['y'])
destination = Point(graph_walk.nodes[walk_node_destination]['x'], graph_walk.nodes[walk_node_destination]['y'])

total_time, m_comb = route_service.route_combined(start, destination)
# Add layer control
folium.LayerControl().add_to(m_comb)

openMap(m_comb)

     shape_id route_id     trip_id     trip_headsign  \
390  MT_L3_r1    MT_L3  MT_L3-2-LV  Arcos de Zapopan   
465    T15_r2      T15  T15_TRIP_2      La Coronilla   

                                        shape_geometry route_short_name  \
390  LINESTRING (-11511679.677 2361368.18, -1151169...               L3   
465  LINESTRING (-11502081.043 2363170.717, -115021...              T15   

                               route_long_name  route_type route_color  \
390  Linea 3 Arcos Zapopan - Central Camionera           0     #B8007C   
465   Troncal T15 - 231 "Prolongacion Alcalde"           3     #D40000   

                                              stop_ids  \
390  [MT-L3-118, MT-L3-117, MT-L3-116, MT-L3-115, M...   
465  [mxa_T15_1_STP_1, mxa_T15_1_STP_2, mxc_C46V1_S...   

                                        stop_headsigns  \
390  [Central De Autobuses, Lázaro Cárdenas, Tlaque...   
465  [Calle Ramón Corona, Calle Pipila, Cuitláhuac,...   

                                

In [ ]:
# Function to trim a LineString shape between two stops
from shapely.geometry import LineString, Point
from shapely.ops import substring

def trim_shape_between_stops(shape_geometry, stops_on_route, stops_df):
    """
    Trim a shape LineString to only include the portion between first and last stop used.
    
    Args:
        shape_geometry: The full LineString of the shape
        stops_on_route: List of stop_ids that are used in this segment
        stops_df: DataFrame with stop information including geometry
    
    Returns:
        Trimmed LineString
    """
    if len(stops_on_route) < 2:
        return shape_geometry
    
    # Get the stop geometries
    first_stop = stops_df[stops_df['stop_id'] == stops_on_route[0]].iloc[0]['geometry']
    last_stop = stops_df[stops_df['stop_id'] == stops_on_route[-1]].iloc[0]['geometry']
    
    # Project stops onto the line to get their position along the line
    first_distance = shape_geometry.project(first_stop)
    last_distance = shape_geometry.project(last_stop)
    
    # Ensure first comes before last (in case route goes backwards)
    start_dist = min(first_distance, last_distance)
    end_dist = max(first_distance, last_distance)
    
    # Extract the substring between the two points
    trimmed = substring(shape_geometry, start_dist, end_dist)
    
    return trimmed


# Example: Trim shapes based on actual stops used
# Assuming you have a route result with stops grouped by shape_id

# Let's say you have a path like: [(stop1, shape1), (stop2, shape1), (stop3, shape2), ...]
# Group stops by shape_id
from collections import defaultdict

def trim_route_shapes(dijkstra_path, transit_df, stops_df):
    """
    Trim shapes to only show the portions actually traveled.
    
    Args:
        dijkstra_path: List of (stop_id, shape_id) tuples from route
        transit_df: DataFrame with shape geometries
        stops_df: DataFrame with stop information
    
    Returns:
        List of trimmed LineStrings with metadata
    """
    # Group stops by shape_id
    shape_stops = defaultdict(list)
    for stop_id, shape_id in dijkstra_path:
        if shape_id != 'walking':  # Skip walking segments
            shape_stops[shape_id].append(stop_id)
    
    trimmed_shapes = []
    
    for shape_id, stops_on_shape in shape_stops.items():
        if len(stops_on_shape) < 2:
            continue
            
        # Get the full shape geometry
        shape_row = transit_df[transit_df['shape_id'] == shape_id].iloc[0]
        full_geometry = shape_row['shape_geometry']
        
        # Trim to actual stops used
        trimmed_geom = trim_shape_between_stops(full_geometry, stops_on_shape, stops_df)
        
        trimmed_shapes.append({
            'geometry': trimmed_geom,
            'shape_id': shape_id,
            'route_name': shape_row['route_long_name'],
            'route_color': shape_row.get('route_color', '#7b1fa2'),
            'stops_used': stops_on_shape
        })
    
    return trimmed_shapes


# Usage example (uncomment when you have a dijkstra_path):
# trimmed = trim_route_shapes(dijkstra_path, transit_df, stops_df)
# trimmed_gdf = gpd.GeoDataFrame(trimmed, crs='EPSG:4326')
# m = trimmed_gdf.explore(column='route_name', cmap='tab20')
# openMap(m)